In [4]:
#for varying windowsize in between 100 and 400 at jumps of 20 and calculating the rmse
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM
from keras.layers import  Dropout


df=pd.read_csv("C:\\Users\\shankhajyoti\\Dropbox\\Deepak\\GOOGL_data.csv")
data_csv=df
 
#how many data we will use 
# (should not be more than dataset length )
data_to_use= len(data_csv)
 
# number of training data
# should be less than data_to_use
train_end =len(data_csv)-459
 
 
total_data=len(data_csv)
 
#most recent data is in the end 
#so need offset
start=total_data - data_to_use
 
 
#currently doing prediction only for 1 step ahead
steps_to_predict =1
 
  
yt = data_csv.iloc [start:total_data ,4]    #Close price
yt1 = data_csv.iloc [start:total_data ,1]   #Open
yt2 = data_csv.iloc [start:total_data ,2]   #High
yt3 = data_csv.iloc [start:total_data ,3]   #Low
vt = data_csv.iloc [start:total_data ,5]    # volume
 
 
print ("yt head :")
print (yt.head())
 
yt_ = yt.shift (-1)
     
data = pd.concat ([yt, yt_, vt, yt1, yt2, yt3], axis =1)
data. columns = ['yt', 'yt_', 'vt', 'yt1', 'yt2', 'yt3']
     
data = data.dropna()
     
print (data)
     
# target variable - closed price
# after shifting
y = data ['yt_']
 
        
#       closed,  volume,   open,  high,   low    
cols =['yt']
x = data [cols]
x_train_initial=x.loc[0:799,:] 
x_test_initial=x.loc[800:1257,:]   
y_train_initial=y.loc[0:799]
y_test_initial=y.loc[800:1257]
min_rmse=float("inf")


for idx in range(5,21):
 scaler_x_train = preprocessing.MinMaxScaler ( feature_range =( -1, 1))
 x_train = np. array (x_train_initial).reshape ((len( x_train_initial) ,len(cols)))
 x_train_copy=x_train
 smoothing_window_size = 20*idx
 for di in range(0,800,smoothing_window_size):
     scaler_x_train.fit(x_train[di:di+smoothing_window_size,:])
     x_train[di:di+smoothing_window_size,:] = scaler_x_train.transform(x_train[di:di+smoothing_window_size,:])

 x_test = np. array (x_test_initial).reshape ((len( x_test_initial) ,len(cols)))
 x_test=scaler_x_train.transform(x_test)

 x_train = x_train.reshape (x_train. shape + (1,)) 
 x_test = x_test.reshape (x_test. shape + (1,))
    

    
    
    
    
    
    
 scaler_y_train = preprocessing. MinMaxScaler ( feature_range =( -1, 1))
 y_train = np.array (y_train_initial).reshape ((len( y_train_initial), 1))
 y_train_copy=y_train
 smoothing_window_size = 20*idx
 for di in range(0,800,smoothing_window_size):
     scaler_y_train.fit(y_train[di:di+smoothing_window_size,:])
     y_train[di:di+smoothing_window_size,:] = scaler_y_train.transform(y_train[di:di+smoothing_window_size,:])

#scaler_y_train.fit(y_train[di+smoothing_window_size:])
#y_train[di+smoothing_window_size:] = scaler_y_train.transform(y_train[di+smoothing_window_size:])
 y_test = np. array (y_test_initial).reshape ((len( y_test_initial) ,1))
#y_test=scaler_y_train.transform(y_test)
 
     

     
     
 

 
 
 seed =2016
 np.random.seed (seed)
 fit1 = Sequential ()
 fit1.add (LSTM (  1000 , activation = 'tanh', inner_activation = 'hard_sigmoid' , input_shape =(len(cols), 1) ))
 fit1.add(Dropout(0.2))
 fit1.add (Dense (output_dim =1, activation = 'linear'))
 
 fit1.compile (loss ="mean_squared_error" , optimizer = "adam")   
 fit1.fit (x_train, y_train, batch_size =18, nb_epoch =25, shuffle = False)
 
 print (fit1.summary())
 

 x_train_copy = np.array(x_train_copy).reshape (x_train_copy. shape + (1,)) 
#x_train = scaler_x_train.inverse_transform (np. array (x_train). reshape ((len( x_train), len(cols)))) 
#y_train = scaler_y_train.inverse_transform (np. array (y_train). reshape ((len( y_train), 1)))    
 pred1 = fit1.predict (x_train_copy) 
#pred1 = scaler_y_train.inverse_transform (np. array (pred1). reshape ((len( pred1), 1)))
 pred1=np.array(pred1).reshape((len(pred1),1))
 sum=0.0
 for i in range(len(y_train)):
     sum=sum+(y_train_copy[i,0]-pred1[i,0])**2

    
 sum1=(sum/len(y_train))**0.5

#x_test = scaler_x_train.inverse_transform (np. array (x_test). reshape ((len( x_test), len(cols))))
#x_test = np.array(x_test).reshape (x_test. shape + (1,)) 
#y_test = scaler_y_train.inverse_transform (np. array (y_test). reshape ((len( y_test), 1)))
 pred1 = fit1.predict (x_test) 
 pred1 = scaler_y_train.inverse_transform (np. array (pred1). reshape ((len( y_test), 1)))
 sum=0.0
 for i in range(len(y_test)):
     sum=sum+(y_test[i,0]-pred1[i,0])**2

    
 sum=(sum/len(y_test))**0.5

 print("window size  rmse_train ",smoothing_window_size,sum1)
 print("window size rmse_test",smoothing_window_size,sum)
 if(min_rmse>=sum):
     min_windowsize=smoothing_window_size
     min_rmse=sum

     
    
 
#fit1.summary()

 

print("window size, min rmse_test ",min_windowsize,min_rmse) 
 



 
 


yt head :
0    393.0777
1    391.6012
2    390.7403
3    391.8214
4    394.3039
Name: close, dtype: float64
             yt        yt_       vt        yt1        yt2        yt3
0      393.0777   391.6012  6031199   390.4551   393.7283   390.1698
1      391.6012   390.7403  4330781   389.5892   391.8915   387.2619
2      390.7403   391.8214  3714176   391.2659   394.3440   390.0747
3      391.8214   394.3039  2393946   390.4551   393.0677   390.3750
4      394.3039   396.8414  3466971   390.2549   394.7644   389.2739
5      396.8414   403.8284  5453980   394.0937   397.0266   393.9285
6      403.8284   396.6262  5857528   398.3930   403.9035   398.0376
7      396.6262   398.1628  5522500   403.0527   404.8895   396.2929
8      398.1628   400.2549  7008464   399.3990   403.1277   396.0056
9      400.2549   395.7804  4103315   400.0296   401.0256   397.2969
10     395.7804   395.4601  4602925   401.5512   404.6092   395.6402
11     395.4601   400.2899  4399988   397.8975   398.3729   392.

C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:115: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(1000, activation="tanh", input_shape=(1, 1), recurrent_activation="hard_sigmoid")`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:117: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:120: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/25
800/800 [==============================] - 15s 18ms/step - loss: 0.1892
Epoch 2/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0411
Epoch 3/25
800/800 [==============================] - 12s 14ms/step - loss: 0.0318
Epoch 4/25
800/800 [==============================] - 14s 17ms/step - loss: 0.0318
Epoch 5/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0319
Epoch 6/25
800/800 [==============================] - 13s 17ms/step - loss: 0.0319
Epoch 7/25
800/800 [==============================] - 14s 17ms/step - loss: 0.0318
Epoch 8/25
800/800 [==============================] - 12s 15ms/step - loss: 0.0321
Epoch 9/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0321
Epoch 10/25
800/800 [==============================] - 12s 15ms/step - loss: 0.0321
Epoch 11/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0320
Epoch 12/25
800/800 [==============================] - 16s 19ms/step - loss: 0.0320
E

C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:115: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(1000, activation="tanh", input_shape=(1, 1), recurrent_activation="hard_sigmoid")`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:117: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:120: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/25
800/800 [==============================] - 17s 22ms/step - loss: 0.1869
Epoch 2/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0337
Epoch 3/25
800/800 [==============================] - 12s 15ms/step - loss: 0.0225
Epoch 4/25
800/800 [==============================] - 12s 14ms/step - loss: 0.0225
Epoch 5/25
800/800 [==============================] - 12s 15ms/step - loss: 0.0226
Epoch 6/25
800/800 [==============================] - 12s 15ms/step - loss: 0.0225
Epoch 7/25
800/800 [==============================] - 12s 16ms/step - loss: 0.0224
Epoch 8/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0225
Epoch 9/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0226
Epoch 10/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0226
Epoch 11/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0225
Epoch 12/25
800/800 [==============================] - 13s 16ms/step - loss: 0.0225
E

...

Epoch 1/25
800/800 [==============================] - 17s 21ms/step - loss: 0.2157
Epoch 2/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0247
Epoch 3/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0101
Epoch 4/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0100
Epoch 5/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0102
Epoch 6/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0102
Epoch 7/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0102
Epoch 8/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0102
Epoch 9/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0103
Epoch 10/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0104
Epoch 11/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0105
Epoch 12/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0107
E

C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:115: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(1000, activation="tanh", input_shape=(1, 1), recurrent_activation="hard_sigmoid")`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:117: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:120: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/25
800/800 [==============================] - 18s 22ms/step - loss: 0.2305
Epoch 2/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0182
Epoch 3/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0081
Epoch 4/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0081
Epoch 5/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0083
Epoch 6/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0086
Epoch 7/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0090
Epoch 8/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0094
Epoch 9/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0098
Epoch 10/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0107
Epoch 11/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0119
Epoch 12/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0129
E

C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:115: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(1000, activation="tanh", input_shape=(1, 1), recurrent_activation="hard_sigmoid")`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:117: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
C:\Users\shankhajyoti\Anaconda2\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:120: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/25
800/800 [==============================] - 18s 23ms/step - loss: 0.2364
Epoch 2/25
800/800 [==============================] - 10s 12ms/step - loss: 0.0114
Epoch 3/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0046
Epoch 4/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0045
Epoch 5/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0045
Epoch 6/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0045
Epoch 7/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0045
Epoch 8/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0046
Epoch 9/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0045
Epoch 10/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0046
Epoch 11/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0046
Epoch 12/25
800/800 [==============================] - 10s 13ms/step - loss: 0.0047
E